# Imports

In [1]:
import cv2, glob, tqdm, os, imutils, json

import numpy as np
import pandas as pd
from sklearn.utils import resample
from skimage.feature import hog, local_binary_pattern
from skimage.morphology import skeletonize

from PIL import Image

import torch
import torch.nn as nn

from geneticalgorithm import geneticalgorithm as ga

/Users/remipetitpierre/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


# Functions

In [2]:
def unroll(list_):
    return [i for item in list_ for i in item]

def cutMapels(image, patch_size: int = 50):
    ''' Cut images into mapels '''
    
    if len(image.shape) > 2:
        dimension = image.shape[2]
    else:
        dimension = 1
        
    img = image.copy()
    shape = img.shape[:2]
    
    # Crop if not divisible by patch size
    if shape[0]%patch_size > 0:
        img = img[:-(shape[0]%patch_size)]
    if shape[1]%patch_size > 0:
        img = img[:, :-(shape[1]%patch_size)]
        
    return img.reshape((shape[0]//patch_size, patch_size, shape[1]//patch_size, patch_size, dimension))

def computeGraphicalLoad(patch):
    ''' Edge-detection-based graphical load '''

    graphical_load, scharr = [], []
    for i in range(3):
        scharr.append((cv2.Scharr(patch, -1, 0, 1)*0.5).astype('uint8') + (cv2.Scharr(
                            patch, -1, 1, 0)*0.5).astype('uint8'))
        graphical_load.append(np.mean(scharr[-1]))
        patch = cv2.resize(patch, (patch.shape[1]//2, patch.shape[0]//2))
    graphical_load = np.sqrt(np.mean(graphical_load))
    
    return graphical_load, scharr[0]

def normalizeOrientation(image, mapel_center):
    ''' Normalize mapel orientation'''

    x, y = mapel_center

    patch = image[x-25:x+25, y-25:y+25]

    gray = cv2.blur(cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY), (3, 3))
    hog_feat = hog(gray, orientations=180, pixels_per_cell=(50, 50), cells_per_block=(1, 1),
                   feature_vector=False)[0,0,0,0]
    
    hog_feat[0] = hog_feat[90] = 0
    angle = np.argmax(hog_feat)

    mapel = imutils.rotate(image[x-36:x+36, y-36:y+36], angle)[11:-11, 11:-11]
    
    return mapel, angle

def getHOG(patch, px_per_cell: int = 5):
    ''' Compute the Histogram of Oriented Gradient (HOG) morphological descriptor '''
    
    hog_feat = hog(patch, orientations=12, pixels_per_cell=(px_per_cell, px_per_cell), cells_per_block=(1, 1),
                   feature_vector=False)[:,:,0,0]
    
    hog_feat = np.sum(np.sum(hog_feat, axis=0), axis=0)
    
    return hog_feat

def getHOG_orient(patch):
    ''' Compute HOG compact orientations '''
    
    hog = getHOG(patch)
    hog = np.concatenate((np.asarray([hog[:,0]]).T, np.asarray([hog[:,12]]).T, 
                    np.asarray([hog[:,6]+hog[:,18]]).T, 
                    np.asarray([hog[:,4]+hog[:,8]+hog[:,16]+hog[:,22]]).T, np.asarray(
        [hog[:,1]+hog[:,2]+hog[:,3]+hog[:,5]+hog[:,7]+hog[:,9]+hog[:,10]+hog[:,11]+hog[:,13]+\
         hog[:,14]+hog[:,15]+hog[:,17]+hog[:,19]+hog[:,21]+hog[:,22]+hog[:,23]]).T), axis=1)
    
    return hog

def getComponentsFeatures(patch):
    ''' Compute topological features '''

    black_pix_n = np.sum(patch == 0)
    skeleton = skeletonize(1-patch.astype('bool'))
    black_skel_n = np.sum(skeleton == 1)
    pix_ratio = black_skel_n/black_pix_n
    _, cc_map = cv2.connectedComponents(skeleton.astype('uint8'))
    _, cc_cnt = np.unique(cc_map, return_counts=True)
    n_cc = np.sum(cc_cnt > 10)
    
    return pix_ratio, n_cc

def getLBPdescriptors(image: np.ndarray, lbp_radius:int = 2, histogram_bins: int = 12):
    ''' Compute Linear Binary Patterns (LBP) texture descriptors'''
    
    lbp = (local_binary_pattern(image, 8*lbp_radius, lbp_radius, 'uniform')).astype('uint8')
    
    return np.histogram(lbp, bins=12, range=(0, 17))[0]

def getColorDescriptors(image: np.ndarray, patch_size:int = 50, histogram_bins: int = 255):
    ''' Compute color moments '''
    
    assert (len(image.shape) == 3), "The image should have 3 color channels"
    
    b, g, r = cv2.split(image)

    color_feat = []
    for color in [b, g, r]:
        features = []
        for i in range((image.shape[0]//patch_size)):
            for j in range((image.shape[1]//patch_size)):
                patch = color[i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size]
                features.append(np.histogram(patch, bins=histogram_bins, range=(0, 255))[0])
        features = np.asarray(features)
        mean = np.argmax(features, axis=1)
        std = np.std(features, axis=1)
        skewness = skew(features, axis=1)
        kurt = kurtosis(features, axis=1)
        color_feat.append(np.array([mean, std, skewness, kurt]))
        
    color_feat = np.concatenate(color_feat).T
    
    return color_feat

def getColorHist(patch: np.ndarray, histogram_bins: int = 12):
    ''' Compute color histogram '''
    
    assert (len(patch.shape) == 3), "The image should have 3 color channels"
    
    b, g, r = cv2.split(patch)

    color_feat = []
    for color in [b, g, r]:
        color_feat.append(np.histogram(color, bins=histogram_bins, range=(0, 255))[0]/(patch.shape[
            0]*patch.shape[1]))
        
    color_feat = np.concatenate(color_feat).T
    
    return color_feat

# Class definition

In [4]:
class Corpus:
    
    def __init__(self, subset, paths):
        
        self.df = subset
        self.arks = subset['ark'].values
        self.scale = [subset['est_scale'].min(), subset['est_scale'].max()]
        self.timespan = [subset['est_date'].apply(np.min).min(), subset['est_date'].apply(np.max).max()]
        
        mask = np.array([(path.split('/')[-1].split('-')[0] in self.arks) for path in paths])
        self.img_paths = np.array(paths)[mask]
        
    def __len__(self):
        
        return len(self.img_paths)

# Code
## Select Corpus
Load the corpus of images and select the subcorpora corresponding to the training sample for the optimization of the feature space

In [6]:
paths = glob.glob('images/*.tif')
len(paths)

10548

In [7]:
df = pd.read_json('metadata.json')
df_paris = df[(df['title_LOC'] == 'Paris')]

melotte = Corpus(df[df['creator'] == 'Melotte'], paths)
berney = Corpus(df[df['creator'] == 'Abram Berney'], paths)
deluz = Corpus(df[df['creator'] == 'Louis Deluz'], paths)

jacoubet = Corpus(df_paris[np.array(['jacoubet' in row.keywords[
    'PER'] for row in df_paris.itertuples()])], paths)
alphand = Corpus(df_paris[np.array(['alphand' in row.keywords[
    'PER'] for row in df_paris.itertuples()])], paths)

ad_rhone = Corpus(df[(df['institution'] == 'AD Rhone') & (df['est_scale'] < 10000)], paths)
ad_carmor = Corpus(df[(df['institution'] == 'AD Cotes Armor') & (df['est_scale'] == 2000)], paths)
ad_hmarne = Corpus(df[(df['institution'] == 'AD Haute Marne') & (df['est_scale'] <= 1500)], paths)

etat_major_40k = Corpus(df[(df['publisher'] == 'Etat Major') & (df['est_scale'] == 40000)], paths)
etat_major_50k = Corpus(df[(df['publisher'] == 'Etat Major') & (df['est_scale'] == 50000)], paths)
swisstopo = Corpus(df[df['institution'] == 'SwissTopo'], paths)

datasets = {'melotte': melotte, 'berney': berney, 'deluz': deluz, 'jacoubet': jacoubet, 'alphand': alphand, 
            'ad_rhone': ad_rhone, 'ad_carmor': ad_carmor, 'ad_hmarne': ad_hmarne, 
            'etat_major_40k': etat_major_40k, 'etat_major_50k': etat_major_50k, 'swisstopo': swisstopo}

# Compute and display the length of the training sample
datasets_len = []
for ds in datasets.values():
    datasets_len.append(len(ds))
np.sum([len(ds) for ds in datasets.values()])

1063

In [15]:
datasets_paths = []
for ds in datasets.values():
    datasets_paths += [path.split('/')[-1][:-4] for path in ds.img_paths]

## Sample mapels and compute figurative features

In [42]:
patch_size = 50
half_size = patch_size//2

patch_indices = np.indices((patch_size, patch_size))
patch_indices = np.reshape(patch_indices, (2, patch_size*patch_size)).T

# Iterate over the image paths
for path in tqdm.tqdm(paths):
    
    # Load image
    image = cv2.imread(path)
    img_name = path.split('/')[-1][:-4]
    
    # Get patches indices
    indices = np.indices((image.shape[0]//patch_size, image.shape[1]//patch_size))*50
    indices = np.array(unroll(np.swapaxes(indices.T, 0, 1)))
    image_border = np.max(indices, axis=0)
    
    # Border mask, corresponding to the map background
    border_mask = (indices[:,0] < image_border[0]) & (indices[:,1] < image_border[1]) & (
        indices[:,0] > 0) & (indices[:,1] > 0)
        
    # Cut candidate patches in the whole image
    img_patches = np.concatenate(np.swapaxes(cutMapels(image, patch_size), 1, 2), axis=0)[border_mask]
    
    # Exclude patches overlapping with the map background
    black_mask = (np.sum(np.sum(np.sum(img_patches, 3) == 0, 2), 1) < 50)
    img_patches, indices = img_patches[black_mask], indices[border_mask][black_mask]
    
    # Compute graphical load
    gload = np.array([computeGraphicalLoad(patch) for patch in img_patches])
    GL_LOAD_AREA = np.sum(gload[:,0] >= 4.4)/len(gload)
    GL_HIST = np.histogram(gload[:,0], range=(0, 12), bins=12)[0]/len(gload)
    
    # Center patch on the most salient graphical element (darkest pixel)
    mapel_centers = np.array([(patch_indices[np.argmax(scharr)]+ind).tolist() for ind, scharr in zip(indices,
                               np.sum(np.array(gload[:, 1].tolist()), axis=3))])
    mapels = np.array([image[center[0]-half_size:center[0]+half_size,
                             center[1]-half_size:center[1]+half_size] for center in mapel_centers])
    
    # Remove background patches again
    black_mask = (np.sum(np.sum(np.sum(mapels, 3) == 0, 2), 1) < 50)
    mapels, mapel_centers = mapels[black_mask], mapel_centers[black_mask]
        
    # Apply graphical load criterium
    gl_feat = np.array([computeGraphicalLoad(mapel)[0] for mapel in mapels])
    gl_mask = (gl_feat >= 4.4)
    mapels, mapel_centers, gl_feat = mapels[gl_mask], mapel_centers[gl_mask], gl_feat[gl_mask]
    
    # Sample the remaining candidate patches (without replacement whenever possible)
    
    sample_ind = np.arange(len(mapel_centers))
    if len(mapel_centers) >= 800:
        sample = resample(sample_ind, replace=False, n_samples=800)
    else:
        sample = resample(sample_ind, replace=True, n_samples=800)
    
    # Normalize the orientation of the mapels
    mapels, mapel_centers = mapels[sample], mapel_centers[sample]
    mapels, ORIENTATION = np.array([normalizeOrientation(image, center) for center in mapel_centers]).T
    
    gray_mapels = [cv2.cvtColor(mapel, cv2.COLOR_RGB2GRAY) for mapel in mapels]
    bin_mapels = [cv2.threshold(mapel.copy(), 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU
                               )[1] for mapel in gray_mapels]
    
    GL_AFTER_REORIENT = np.array([computeGraphicalLoad(mapel)[0] for mapel in mapels])
    
    LBP_2 = np.array([getLBPdescriptors(mapel, lbp_radius=2)[0] for mapel in mapels])
    LBP_3 = np.array([getLBPdescriptors(mapel, lbp_radius=3)[0] for mapel in mapels])
    LBP_4 = np.array([getLBPdescriptors(mapel, lbp_radius=4)[0] for mapel in mapels])
    
    COLOR_DESC = np.array([getColorDescriptors(mapel)[0] for mapel in mapels])
    COLOR_HIST_6 = np.array([getColorHist(mapel, 6) for mapel in mapels])
    COLOR_HIST_9 = np.array([getColorHist(mapel, 9) for mapel in mapels])
    
    HOG_BIN_ORIENT = np.array([getHOG_orient(mapel) for mapel in mapels])[:,0]
    HOG_5 = np.array([getHOG(mapel, 5) for mapel in mapels])
    HOG_10 = np.array([getHOG(mapel, 10) for mapel in mapels])
    HOG_25 = np.array([getHOG(mapel, 25) for mapel in mapels])
    
    COMP = np.array([getComponentsFeatures(mapel) for mapel in bin_mapels])
    
    features = np.concatenate([ORIENTATION.reshape(-1,1),
                               HOG_BIN_ORIENT,
                               GL_AFTER_REORIENT.reshape(-1,1),
                               LBP_2, LBP_3, LBP_4,
                               COLOR_DESC, COLOR_HIST_6, COLOR_HIST_9,
                               HOG_5, HOG_10, HOG_25,
                               COMP], axis=1)
        
    # Save patches
    cv2.imwrite(f'mapels/{img_name}.tif', np.concatenate(mapels, axis=0))
    
    # Save mapels features
    np.save(f'features/{img_name}.npy', features)

  0%|          | 0/10548 [00:00<?, ?it/s]<ipython-input-42-402b1d6583f5>:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  gload = np.array([computeGraphicalLoad(patch) for patch in img_patches])
<ipython-input-42-402b1d6583f5>:54: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  texels, ORIENTATION = np.array([normalizeOrientation(image, center) for center in texel_centers]).T
100%|██████████| 10548/10548 [40:48:01<00:00, 13.93s/it]    


## Optimize feature weights

In [ ]:
device = torch.device('cuda:0')

best_score = -100.

def f(X):
    
    ''' Objective function '''
    
    global best_score
    
    # Features description
    feat_structure = {
        'ORIENTATION': {'norm': np.array([179.])/X[0], 'active': True},
        'HOG_BIN_ORIENT': {'norm': [72., 72., 72., 72., 72.*4], 'active': False},
        'GL_AFTER': {'norm': np.array([12.])/X[1], 'active': True},
        'LBP_2': {'norm': np.array(12*[1.]), 'active': False},
        'LBP_3': {'norm': np.array(12*[1.]), 'active': False},
        'LBP_4': {'norm': np.array(12*[1.])/X[2], 'active': True},
        'COLOR_DESC': {'norm': np.array(3*[254.*np.inf, 153.*np.inf, 16.*np.inf, 250.*np.inf]), 'active': False},
        'COLOR_HIST_6': {'norm': 18*[1.], 'active': False},
        'COLOR_HIST_9': {'norm': 27*[1.], 'active': False},
        'HOG_5': {'norm': 12*[484/100], 'active': False},
        'HOG_10': {'norm': np.array(12*[484/25]), 'active': False},
        'HOG_25': {'norm': np.array(12*[484/4])/X[3], 'active': True},
        'NOTSU': {'norm': [2500.], 'active': False},
        'SKEL_RATIO': {'norm': np.array([1.])/X[4], 'active': True},
        'NCOMP': {'norm': np.array([21.])/X[5], 'active': True},
    }

    # Create normalization vector and features mask
    normalization = np.array(unroll([feat_structure[key]['norm'] if feat_structure[key][
        'active'] else [] for key in feat_structure.keys()]))

    mask = np.array(unroll([len(feat_structure[key]['norm'])*[True] if feat_structure[key][
        'active'] else len(feat_structure[key]['norm'])*[False] for key in feat_structure.keys()]))

    # Subset paths corresponding to the training subset
    with open('subset_paths.json', 'r') as f:
        subset_paths = json.load(f)

    # Load image features
    image_features = torch.Tensor([(np.load(f'features/{img_name}.npy', allow_pickle=True)[:, mask]
                                   )/normalization for img_name in subset_paths]).to(device)
    n = image_features.shape[0]
    
    # Compute distance matrices (Equation 2)
    K = np.arange(0.0, 0.1, 0.001)
    matrix = []
    for i in range(n):
        dist_matrix = torch.cdist(image_features[i], image_features)
        dist_0 = dist_matrix.min(dim=1).values
        matrix.append(torch.cat([(dist_0 <= k).sum(dim=1) for k in K]).view(len(K), -1).T.true_divide(800))
    matrix = torch.cat(matrix).view(n, n, len(K)).cpu()

    # Classes encoding
    datasets_len = [10, 10, 21, 56, 48, 116, 289, 329, 20, 22, 142]
    classes = []
    for i, N in enumerate(datasets_len):
        classes += N*[i]
    classes = np.array(classes)

    largest_dist = -1
    best_params = ()
    best_matrix = None
    
    # Compute optimal value of k, the radius of free variation, for this features set
    for k in range(len(K)):
        matrix_ = torch.cat([matrix[:,:,k], matrix[:,:,k].T]).view(2, n, n).mean(0).numpy()
        np.fill_diagonal(matrix_, np.nan)

        interclass, interclass_std = [], []
        class_matrix, class_matrix_std = np.zeros((11, 11)), np.zeros((11, 11))
        for i in range(11):
            for j in range(11):
                class_matrix[i, j] = np.nanmedian(matrix_[classes == i][:,classes == j])
                class_matrix_std[i, j] = np.nanstd(matrix_[classes == i][:,classes == j])
                if i != j:
                    interclass.append(class_matrix[i, j])
                    interclass_std.append(class_matrix_std[i, j])

        intraclass = np.diag(class_matrix)-np.diag(class_matrix_std)

        # Equation 3
        stat_dist = np.abs(np.mean(np.diag(class_matrix)) - np.mean(interclass))/(
            0.5*np.mean(np.diag(class_matrix_std)) + 0.5*np.mean(interclass_std))

        # Save the best solution to Eq. 3 given the different values of k
        if stat_dist > largest_dist:
            largest_dist = stat_dist
            best_params = (l, K[k])
            best_matrix = matrix_.copy()
                
    # Save the best solution
    if largest_dist > best_score:
        best_score = largest_dist
        image = Image.fromarray((best_matrix*255).astype('uint8'))
        image.save(f'optim/best_matrix.png', 'PNG')
        with open('optim/best_matrix.npy', 'wb') as f:
            np.save(f, best_matrix)

    return 10 - largest_dist


# Research space for the weights
boundaries = np.array([6*[0.01, 1.0]]).reshape(6, 2)

# Parameters of the genetic algorithm
algorithm_param = {'max_num_iteration': 80,
                   'population_size': 20,
                   'mutation_probability': 0.2,
                   'elit_ratio': 0.1,
                   'crossover_probability': 0.5,
                   'parents_portion': 0.2,
                   'crossover_type': 'uniform',
                   'max_iteration_without_improv': 20}

# Genetic algorithm
model = ga(function=f, dimension=6, variable_type='real', variable_boundaries=boundaries,
           algorithm_parameters=algorithm_param, convergence_curve=False)
model.run()

# Best solution
X = np.array([0.05, 0.15, 0.1, 1.0, 0.25, 0.25])
f(X)